# Deep Feed Forward Networks

In [1]:
import numpy as np 
import tensorflow as tf

## Network Construction Phase

In [2]:
num_inputs = 28*28 # MNIST
h1 = 300 # hidden layer 1,2 size
h2 = 100 
num_out = 10

### Set Placeholders

In [3]:
X = tf.placeholder(tf.float32, shape=(None, num_inputs), name='X')
y = tf.placeholder(tf.int64, shape=(None, 10), name='y')

### Set Function For Neuron Layer

In [4]:
def neuron_layer(X, num_neurons, name, activation= None):
    with tf.name_scope(name):
        
        num_inputs = int(X.get_shape()[1])
        variance = 2 / np.sqrt(num_inputs)
        
        W = tf.Variable(tf.truncated_normal((num_inputs, num_neurons), stddev=variance), name='weights')
        b = tf.Variable(tf.zeros([num_neurons]), name='bias')
        
        z = tf.add(tf.matmul(X, W), b)
        
        if activation is None:
            return z

        activation = activation.lower()
        if activation == 'relu':
            return tf.nn.relu(z)
        elif activation == 'sigmoid':
            return tf.nn.sigmoid(z)
        elif activation == 'softmax':
            return tf.nn.softmax(z)

### Create all layers of the network

In [5]:
with tf.name_scope('dnn'):
    hidden1 = neuron_layer(X, h1, name='hidden1', activation='ReLU')
    hidden2 = neuron_layer(hidden1, h2, name='hidden2',activation='ReLU')
    out = neuron_layer(hidden2, num_out, name='out', activation='softmax')

Instructions for updating:
Colocations handled automatically by placer.


### Define Loss

In [6]:
with tf.name_scope('loss'):
    entropy = tf.losses.softmax_cross_entropy(onehot_labels=y, logits=out)
    loss = tf.reduce_mean(entropy, name='loss')

Instructions for updating:
Use tf.cast instead.


### Define optimizer

In [7]:
eta = 1e-2

with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=eta)
    train_step = optimizer.minimize(loss)

### Set Evaluation Method

In [8]:
with tf.name_scope('eval'):

    predictions = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(predictions, tf.float32))

### Define Initializer and Saver

In [9]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

## LOAD MNIST AND TRAIN NETWORK

In [11]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./tmp/data')

Extracting ./tmp/data/train-images-idx3-ubyte.gz
Extracting ./tmp/data/train-labels-idx1-ubyte.gz
Extracting ./tmp/data/t10k-images-idx3-ubyte.gz
Extracting ./tmp/data/t10k-labels-idx1-ubyte.gz


### Train

In [ ]:
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(30):
        for iteration in range(mnist.train.num_examples // batch_size):
            
            # get next batch
            X_batch_temp, y_batch_temp = mnist.train.next_batch(batch_size)
            X_batch = np.reshape(X_batch_temp, [-1, 784])
            y_batch = np.eye(10)[y_batch_temp] # numpy one-hot encoding
            
            # execute update step 
            sess.run(train_step, feed_dict={X: X_batch, y: y_batch})
            
        # Evaluate accuracy on test set
        test_acc = sess.run(accuracy, feed_dict={X: mnist.test.images, y:np.eye(10)[mnist.test.labels]})
        
        print(f'Test set accuracy in epoch {epoch+1}: {round(test_acc*100, 2)}%')
            
    # save model
    save_path = saver.save(sess, 'final_model.ckpt')

Test set accuracy in epoch 1: 63.66%
Test set accuracy in epoch 2: 72.77%
Test set accuracy in epoch 3: 74.07%
Test set accuracy in epoch 4: 74.7%
Test set accuracy in epoch 5: 75.04%
Test set accuracy in epoch 6: 75.27%
Test set accuracy in epoch 7: 75.38%
Test set accuracy in epoch 8: 75.6%
Test set accuracy in epoch 9: 77.51%
Test set accuracy in epoch 10: 83.15%
Test set accuracy in epoch 11: 83.59%
Test set accuracy in epoch 12: 83.9%
